# Training and Detection with YOLOv5

## Aim

The notebook explains YOLOv5 model train with the annotated shorcut detection dataset, evaluation of the trained model with performance metrices, and detection on given images with defined outputs: images with detection boxes and text files detection data.

## Preliminaries

* The working directory contains YOLO v5 model and datasets.
* For this application it has chosen the smallest, fastest base model of YOLOv5, YOLOv5s. 
    * This model has been selected because the application deals with a single class and the complexity of the visual object to be detected is relatively easier compared to COCO dataset class objects.
* The pre-trained YOLOv5 model weights trained on COCO 128 dataset are used for transfer learning (can be found [here](https://github.com/ultralytics/yolov5/releases)).
* After training The trained weights, images with boundin boxes after detection, and evaluation metrices results are also stored in the ``` yolov5/runs/train ``` directory.
* After detection images with bounding boxes, text files with detecion data can be found in the ``` yolov5/runs/detect ``` directory.

## Data

* Datasets for training and detection can be found in the ```datsets``` directory of the repository and also in the here in the cloud.
* Detection results can also be found here in the cloud here.

## Tasks

* Setup environmnet and install dependecies to run YOLOv5.
* Select important hyperparameters.
    * Batch size, number of epochs, output image size are defined based on the size of the dataset.   
* Train the YOLOv5 model with pretrained weights.
* Evaluate the training with Tensorboard.
* Detection for given images with other detecion data.


## Results

* Trained model weights for detection of shortcuts which can be used for detection.
* Detection outcome for a given image.
    * Images with marked bounding boxes.
    * Text file with detction data.





## Setup and install

### Install dependencies

In [1]:
# change working directory to yolo5 dir
%cd yolov5
%pip install -qr requirements.txt # install dependencies

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-pc/code/Users/D.DESILVA/object_detection_with_yolov5/Object_detection_model_v1_Train/yolov5
Note: you may need to restart the kernel to use updated packages.
Setup complete. Using torch 1.8.1+cu102 (Tesla K80)


## YOLOv5 model configuration for training

The model-configurations file dictates the model architecture. The original model implementation provides several models with model configuration files. These configuration files can be found in ```models``` directory. 
Since the smallest, fastest base model YOLOv5s is selected and the regarding 
Configuration file ```YOLOv5s.yaml``` is used during the training as YOLOv5s is the base model for this application. Since the training is initialized from from pretrained weights as in this case, there is no need to edit the model-configurations file because the model will be extracted with the pretrained weights. 

However if the model needs to train from the scratch, the configuration file needs to be modified according to the requirement of the application by just editing the number of classes ```nc``` parameter to the number of classes in the dataset (single class ```nc```= 1)


## Train YOLOv5 model on the custom dataset

Arguments for training script:

* ```img```: Define input image size (480).
* ```batch```: Determine batch size (This depends on size of the dataset).
* ```epochs```: Define the number of training epochs (this depends on the size of the dataset and the complexity of the dataset).
* ```data```: The dataset locaiton is saved in the dataset.location (.yaml file is directed here).
* ```weights```: Specify a path to weights to start transfer learning from. Here it has chosen the generic COCO pretrained checkpoint.
* ```cache```: cache images (ram) for faster training.

### Hyper parameters for training

The hyperparameters-configurations file defines 28 hyperparameters for the training. This includes the learning rate, momentum, losses, augmentations etc. This open source YOLOv5 development provides a default hyperparameters file under the ````data/hyp/hyp.scratch.yaml```` directory and ````run/train/exp/hyp.yaml````  after training. It has started the training in this case with default hyperparameters to establish a performance baseline.


In [2]:
!python train.py --img 480 --batch 25 --epochs 100 --data data_7.yaml --weights ../pretrained/yolov5s.pt --cache

train: weights=../pretrained/yolov5s.pt, cfg=, data=data_7.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=25, imgsz=480, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
fatal: not a git repository (or any parent up to mount point /mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-pc)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
YOLOv5 🚀 2022-6-9 torch 1.8.1+cu102 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_dec

## YOLOv5 model performance evaluation metrics


**01. Mean Average Precision (mAP)**

Mean Average Precision (mAP) is one of the most popular metrics used to evaluate object detection models. The calculation of mAP requires Intersect Over Union (IOU), Precision, Recall, Precision-Recall Curve, and AP´. Understanding mAP can start with IOU and IOU indicates how much given two bounding boxes overlap. When calculating mAP, it mainly determines how much overlap of detection and ground truth bounding boxes. For this purpose, IOUs can be used, and mAP50 is defined as the accuracy when IOU = 50. This means that if there is more than 50% overlap of detection and ground truth, the detection is considered successful. The larger the IOU, the more accurate the bounding box needs to be detected to match with the ground truth. Furthermore, the overlap can be increased for this mean average precision such as mAP75 with 75% overlap. Therefore, it will be harder if one increases the overlap such as mAP75 is often lower than the value of mAP50.

**02. Precision** 

Precision is the ability of the model to detect only the relevant objects. It measures how many of the bounding box predictions or positive identifications are correct. 

**03. Recall**

The recall is the ability of a model to find all ground truth bounding boxes. It answers the question What proportion of true positives among all ground truths is identified correctly. A model is said to be a good model if it has high precision and high recall.

**04. Precision-Recall Curve**

The Precision-Recall Curve is a plot of Precision on the vertical axis and Recall on the horizontal axis. It can say that for a good model, precision and recall stays high even when the confidence score is varied.

**05. Loss metrics**

The Tensorborad (from training) provides several validation metrics and losses are among them. The YOLO loss function is composed of three parts:

* **box loss** : bounding box regression loss (Mean Squared Error).
* **object loss** : the confidence of object presence in the objectness loss (Binary Cross Entropy).
* **class loss** : the classification loss (Cross-Entropy).

Since the dataset has one class only, there are no class misidentifications, and the classification error is constantly zero.

In [7]:
# Start tensorboard
# Launch after started training
# logs save in the directory "runs"
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Launching TensorBoard...

## Validation

In [10]:
# newly annotaed dataset (100 epoch)
!python val.py --weights runs/train/exp12/weights/best.pt --data data_5.yaml --img 480 --iou 0.5 --half --conf .3

val: data=/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-pc/code/Users/D.DESILVA/object_detection_with_yolov5/Object_detection_model_v1_Train/yolov5/data/data_5.yaml, weights=['runs/train/exp12/weights/best.pt'], batch_size=32, imgsz=480, conf_thres=0.3, iou_thres=0.5, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
fatal: not a git repository (or any parent up to mount point /mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-pc)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
YOLOv5 🚀 2022-6-9 torch 1.8.1+cu102 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning '/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-pc/code/Users
               Class     Images     Labels          P          R     mAP@.5 mAP@
   

## Run detection with trained weights

Run detection with a pretrained checkpoint on contents of test images or any given images directed.

Arguments for detection script:

* ```weights```: Trained weights location
* ```img```: Image size
* ```conf```: Specifying model confidence. Higher confidence required makes less prediction. The least confidance a detection should have to classify as an object
* ```source```: detection images directory
* ```hide-labels```: Hide labels and confidance as this case is a single class object detection problem
* ```line-thickness```: Boundry box line thickness
* ```save-text```: Save detection info to a text. In this case boundary box coordinates (top left & bottom right)
* ```save-conf```: Save detection confidance to text file.


In [3]:
!python detect.py --weights runs/train/exp13/weights/best.pt --img 480 --conf 0.3 --source ../datasets/ELWO_IR_dataset_7/valid/images --hide-labels --line-thickness 1 --save-txt --save-conf

detect: weights=['runs/train/exp13/weights/best.pt'], source=../datasets/ELWO_IR_dataset_7/valid/images, data=data/coco128.yaml, imgsz=[480, 480], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=True, hide_conf=False, half=False, dnn=False
fatal: not a git repository (or any parent up to mount point /mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-pc)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
YOLOv5 🚀 2022-6-9 torch 1.8.1+cu102 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/15 /mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-pc/code/Users/D.DESILVA/object_detection_with_yolov5/Object_detection_model_v1_Train/datasets/ELWO_IR_datas

### Detection results

**IR images with bounding boxes**

These bounding boxes do not contain class label and detection confidence as the task in hand is a single label case and having such details may clutter the image with data respectively.

![outpu%20from%20detection.PNG](attachment:outpu%20from%20detection.PNG)

**Text files with detection data**

![labels.PNG](attachment:labels.PNG)